<a href="https://colab.research.google.com/github/jyryu3161/DrugDiscovery/blob/main/Lec9_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Virtual Screening Using Molecular Docking

# Installation

In [ ]:
!git clone https://github.com/jyryu3161/DrugDiscovery.git
!chmod u+x ./DrugDiscovery/qvina2.1
!chmod u+x ./DrugDiscovery/vina

Cloning into 'DrugDiscovery'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 212 (delta 32), reused 0 (delta 0), pack-reused 144 (from 1)
Receiving objects: 100% (212/212), 4.91 MiB | 8.75 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [ ]:
!pip install -q condacolab # install the condacolab package
import condacolab # Import and initialize condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...


In [ ]:
import condacolab
condacolab.check() # verification of the installation

✨🍰✨ Everything looks OK!


In [ ]:
# Create a new environment (optional)
!conda create -n myenv python=3.9 -y
# Install packages
!conda install -c conda-forge matplotlib rdkit -y
!pip install --pre deepchem
!pip install ogb
!pip install py3Dmol # 3D Molecular Visualizer
!pip install oddt

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/myenv

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.4.26  |       hbd8a1cb_0         149 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 KB  conda-forge
    libexpat-2.7.0             |       h5888daf_0          73 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-15.1.0              |       h767d61c_2         810 KB  conda-forge
    libgcc-ng-15.1.0           |       h69a702a_2          34 KB  conda-forge
    lib

# Input setting

In [ ]:
protein_pdb_file = './DrugDiscovery/STK33_alphafold.pdb'
center = (11.5518, 1.2844, 2.5799) # 예측된 binding site를 넣을 것 - x, y, z
smiles = 'COc1ccc(cc1)-n1ncc2ccc(Nc3ccc(N4CCN(C)CC4)c(OC)c3)nc12'
docking_output_dir = './output_ref/'


# Run molecular docking

In [ ]:
import py3Dmol
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

def make_3d_coordinate_test(m):
    target_mols = []
    m = Chem.AddHs(m)
    potential = AllChem.ETKDG()
    AllChem.EmbedMolecule(m, potential)
    cids = AllChem.EmbedMultipleConfs(m, numConfs=1, numThreads=15)

    for conf in m.GetConformers():
        tm = Chem.Mol(m,False,conf.GetId())
        res = AllChem.MMFFOptimizeMoleculeConfs(tm)
        target_mols.append(tm)
    return target_mols[0]


In [ ]:
import os
import time

try:
    os.mkdir(docking_output_dir)
except:
    pass

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import oddt
from oddt.docking.AutodockVina import autodock_vina
from oddt.docking import AutodockVina

s = time.time()

mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

mol = make_3d_coordinate_test(mol)

with Chem.SDWriter('molecule.sdf') as w:
    w.write(mol)


protein_file = protein_pdb_file

protein = next(oddt.toolkit.readfile('pdb', protein_file))
protein.protein = True

mol = Chem.MolFromMolFile('molecule.sdf')
mol2 = Chem.MolToMolBlock(mol)
dock_mol = oddt.toolkit.readstring('sdf', mol2)

protein.addh(only_polar=True)

vina_obj = autodock_vina(protein=protein, center=center, size=(20, 20, 20), num_modes=3, exhaustiveness=8, executable='./DrugDiscovery/qvina2.1') # vina

docking_outputs = vina_obj.dock([dock_mol])
docking_cnt = 1
for each_output in docking_outputs:
    vina_affinity = each_output.data['vina_affinity']
    AutodockVina.write_vina_pdbqt(each_output, docking_output_dir, name_id='ref_docking_output')

    print ("%s docking pose's binding energy (kcal/mol) : %s"%(docking_cnt, vina_affinity))
    docking_cnt+=1
    break
e = time.time()

print(e-s)

1 docking pose's binding energy (kcal/mol) : -8.5
107.49835705757141


# Visualization

In [ ]:
def drawit2(m,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p

def DrawComplex(protein, ligand):
    complex_pl = Chem.MolToPDBBlock(Chem.CombineMols(protein, ligand))
    view = py3Dmol.view(width=600, height=600)
    view.addModel(complex_pl, 'pdb')

    view.setStyle({}, {'cartoon': {'color': 'spectrum'}})
    view.setStyle({'resn': 'UNL'}, {'stick': {'colorscheme': 'default'}})
    view.addSurface(py3Dmol.VDW, {'opacity': 0.6, 'color': 'orange'}, {'resn': 'UNL'})

    view.zoomTo()
    return view

filename = './output_ref/ref_docking_output.pdbqt'

ligand_oddt = oddt.toolkit.readfile('pdbqt', filename).__next__()

ligand_mol_block = ligand_oddt.write('mol')
ligand = Chem.MolFromMolBlock(ligand_mol_block, removeHs=False)

protein = Chem.MolFromPDBFile(protein_pdb_file)

DrawComplex(protein, ligand)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# HIgh-Throughput Virtual Screening

In [ ]:
import pandas as pd
import os
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import oddt
from oddt.docking.AutodockVina import autodock_vina
from oddt.docking import AutodockVina
import glob
import tqdm
import numpy as np

# Function to generate 3D coordinates for a molecule
def make_3d_coordinate(m, conf_num):
    target_mols = []
    m = Chem.AddHs(m)
    potential = AllChem.ETKDG()
    AllChem.EmbedMolecule(m, potential)
    cids = AllChem.EmbedMultipleConfs(m, numConfs=conf_num, numThreads=15)

    for conf in m.GetConformers():
        tm = Chem.Mol(m,False,conf.GetId())
        res = AllChem.MMFFOptimizeMoleculeConfs(tm)
        target_mols.append(tm)
    return target_mols

def get_3d_conformers(smiles, conf_num=3):
    mol = Chem.MolFromSmiles(smiles)
    target_mols = make_3d_coordinate(mol, conf_num)

    target_dock_mols = []
    for mol3d_tmp in target_mols:
        mol3d_tmp = Chem.MolToMolBlock(mol3d_tmp)
        dock_mol = oddt.toolkit.readstring('sdf', mol3d_tmp)
        target_dock_mols.append(dock_mol)
    return target_dock_mols

target_folder_dir = './inputs/'
output_dir = './output_vs/'

each_target_dir = './target_dir/'

basename = os.path.basename(each_target_dir)

protein_pdb_file = each_target_dir+ '/protein.pdb'
ref_docking_file = each_target_dir+ '/ref_docking_output.pdbqt'
lib_file = each_target_dir+'/lib.csv'

# Create output directory
docking_output_dir = output_dir+"%s"%(basename)
try:
    os.makedirs(docking_output_dir, exist_ok=True)
except:
    pass

# Read the CSV file
csv_file = lib_file
df = pd.read_csv(csv_file)
df = df[0:5]

# Parameters
protein_pdb_file = protein_pdb_file
top_n = 100  # Number of top results to output
exhaustiveness = 8
num_modes = 3
conformer_num = 2

# Prepare protein
protein = next(oddt.toolkit.readfile('pdb', protein_pdb_file))
protein.protein = True
protein.addh(only_polar=True)

ligand = next(oddt.toolkit.readfile('pdbqt', ref_docking_file))
ligand.protein = False
ligand.addh(only_polar=True)

# Initialize Vina
vina_obj = autodock_vina(protein=protein, auto_ligand=ligand, size=(20, 20, 20), num_modes=3, exhaustiveness=16, executable='./DrugDiscovery/qvina2.1')

# Store docking results
docking_results = []

# Process each molecule
for index, row in tqdm.tqdm(df.iterrows()):
    smiles = row['SMILES']
    catalog_id = row['Catalog ID']

    # Convert SMILES to RDKit molecule
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Failed to parse SMILES for {catalog_id}")
        continue

    docking_mols = get_3d_conformers(smiles, conformer_num)

    # Perform docking
    docking_outputs = vina_obj.dock(docking_mols)

    vina_affinity_list = []
    pose_outputs = []

    for idx, output in enumerate(docking_outputs, 1):
        vina_affinity = output.data['vina_affinity']
        vina_affinity_list.append(float(vina_affinity))
        pose_outputs.append((output, idx))

    # Find the best pose (lowest affinity)
    if vina_affinity_list:  # Ensure there are results
        min_vina_affinity = np.min(vina_affinity_list)
        best_pose_idx = np.argmin(vina_affinity_list) + 1  # 1-based index
        best_output = pose_outputs[best_pose_idx - 1][0]  # Get the best pose output

        print(f"{catalog_id}: Best Vina Affinity = {min_vina_affinity} kcal/mol (Pose {best_pose_idx})")

        # Save only the best docking pose
        output_file = f'docking_pose_{catalog_id}_{best_pose_idx}.pdbqt'
        AutodockVina.write_vina_pdbqt(
            best_output,
            docking_output_dir,
            name_id=f'docking_pose_{catalog_id}_{best_pose_idx}'
        )

        # Store the best pose result
        docking_results.append({
            'Catalog ID': catalog_id,
            'SMILES': smiles,
            'Vina Affinity (kcal/mol)': min_vina_affinity,
            'Pose': best_pose_idx,
            'Output File': output_file
        })

# Sort results by binding affinity (more negative is better)
docking_results = sorted(docking_results, key=lambda x: x['Vina Affinity (kcal/mol)'])

# Save results to CSV
results_df = pd.DataFrame(docking_results)
results_df.to_csv(os.path.join(docking_output_dir, "docking_results.csv"), index=False)
print(f"\nAll docking results saved to {docking_output_dir}/docking_results.csv")

1it [01:56, 116.63s/it]

Z384217462: Best Vina Affinity = -6.5 kcal/mol (Pose 4)


2it [03:28, 102.30s/it]

Z2599525455: Best Vina Affinity = -6.6 kcal/mol (Pose 1)


3it [05:47, 119.01s/it]

Z1554054473: Best Vina Affinity = -7.6 kcal/mol (Pose 1)


4it [08:59, 147.89s/it]

Z1322679732: Best Vina Affinity = -7.5 kcal/mol (Pose 1)


5it [11:19, 135.87s/it]

Z1367139877: Best Vina Affinity = -7.1 kcal/mol (Pose 1)

All docking results saved to ./output_vs/20210378/docking_results.csv
